In [ ]:
import pygromos
from pygromos import files
from pygromos.files.topology import *
from pygromos.files.blocks.topology_blocks import *
from rdkit import Chem
from rdkit.Chem import AllChem
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines import smirnoff
from pygromos import data
from pygromos.files.gromos_system import *
from simtk.unit import *


In [ ]:
ff = data.data_dir + "/ff/SMIRNOFF/frosst_12-12-18.xml"
forcefield = smirnoff.ForceField(ff)

In [ ]:
rdkmol=Chem.MolFromMolFile("/home/mlehner/Documents/dev/molecule_174.sdf", removeHs = False)

In [ ]:
moleculeFF = Molecule.from_rdkit(rdkmol)
topology = Topology.from_molecules(moleculeFF)

In [ ]:
moleculeFF.name

In [ ]:
molecule_force_list = forcefield.label_molecules(topology)

In [ ]:
top = Top(input=data.data_dir+"/topology_templates/blank_template.top")

In [ ]:
for molecule in molecule_force_list:
    for key in molecule["Bonds"]:
        force = molecule["Bonds"][key]
        #hQ = topology.atom(force[0]).atomic_number == 1 or topology.atom(force[1]).atomic_number == 1
        hQ =  not all([topology.atom(x).atomic_number != 1 for x in key]) 
        top.add_new_bond(k=force.k.value_in_unit(kilojoule / (mole * nanometer ** 2)), b0=force.length.value_in_unit(nanometer), atomI=key[0]+1, atomJ=key[1]+1, includesH=hQ)

In [ ]:
for molecule in molecule_force_list:
    for key in molecule["Angles"]:
        force = molecule["Angles"][key]
        hQ =  not all([topology.atom(x).atomic_number != 1 for x in key])
        top.add_new_angle(k=force.k.value_in_unit(kilojoule / (mole * radian ** 2)), b0=force.angle.value_in_unit(degree), atomI=key[0]+1, atomJ=key[1]+1, atomK=key[2]+1, includesH=hQ)

In [ ]:
for molecule in molecule_force_list:
    for key in molecule["ProperTorsions"]:
        force = molecule["ProperTorsions"][key]
        hQ =  not all([topology.atom(x).atomic_number != 1 for x in key])
        atomI=key[0]+1
        atomJ=key[1]+1
        atomK=key[2]+1
        atomL=key[3]+1
        k_list = force.k
        phase_list = force.phase
        per_list = force.periodicity
        for t in range(len(k_list)):
            CP=k_list[t].value_in_unit(kilojoule/mole)
            PD=phase_list[t].value_in_unit(degree)
            NP=per_list[t]
            top.add_new_torsiondihedral(CP=CP, PD=PD, NP=NP, atomI=atomI, atomJ=atomJ, atomK=atomK, atomL=atomL, includesH=hQ)

In [ ]:
for molecule in molecule_force_list:
    for key in molecule["ImproperTorsions"]:
        force = molecule["ImproperTorsions"][key]
        hQ =  not all([topology.atom(x).atomic_number != 1 for x in key])
        atomI=key[0]+1
        atomJ=key[1]+1
        atomK=key[2]+1
        atomL=key[3]+1
        k_list = force.k
        phase_list = force.phase
        per_list = force.periodicity
        for t in range(len(k_list)):
            CP=k_list[t].value_in_unit(kilojoule/mole)
            PD=phase_list[t].value_in_unit(degree)
            NP=per_list[t]
            top.add_new_torsiondihedral(CP=CP, PD=PD, NP=NP, atomI=atomI, atomJ=atomJ, atomK=atomK, atomL=atomL, includesH=hQ)
if not hasattr(top, "IMPDIHEDRALTYPE"):
    top.add_block(blocktitle="IMPDIHEDRALTYPE", content=[])
if not hasattr(top, "IMPDIHEDRALH"):
    top.add_block(blocktitle="IMPDIHEDRALH", content=[])
if not hasattr(top, "IMPDIHEDRAL"):
    top.add_block(blocktitle="IMPDIHEDRAL", content=[])

In [ ]:
exclusionlist=dict()

In [ ]:
exclusionlist

In [ ]:
exclusionlist=dict()
for molecule in molecule_force_list:
    for key in molecule["Bonds"]:
        if not str(key[0]) in exclusionlist.keys():
            exclusionlist[str(key[0])] = {key[1]}
        exclusionlist[str(key[0])].add(key[1])
    for key in molecule["Angles"]:
        if not str(key[0]) in exclusionlist.keys():
            exclusionlist[str(key[0])] = {key[1]}
        exclusionlist[str(key[0])].add(key[1])
        exclusionlist[str(key[0])].add(key[2])
    for key in molecule["ProperTorsions"]:
        if not str(key[0]) in exclusionlist.keys():
            exclusionlist[str(key[0])] = {key[1]}
        exclusionlist[str(key[0])].add(key[1])
        exclusionlist[str(key[0])].add(key[2])
        exclusionlist[str(key[0])].add(key[3])
    for key in molecule["ImproperTorsions"]:
        if not str(key[0]) in exclusionlist.keys():
            exclusionlist[str(key[0])] = {key[1]}
        exclusionlist[str(key[0])].add(key[1])
        exclusionlist[str(key[0])].add(key[2])
        exclusionlist[str(key[0])].add(key[3])

In [ ]:
exclusionlist

In [ ]:
moleculeFF.atoms[0].mass.value_in_unit(dalton)

In [ ]:
moleculeFF.atoms[0].atomic_number

In [ ]:
moleculeItr = 1
for molecule in molecule_force_list:
    for key in molecule["vdW"]:
        force = molecule["vdW"][key]
        ATNM = int(key[0]) + 1
        MRES = moleculeItr
        IAC = 0
        MASS = moleculeFF.atoms[int(key[0])].mass.value_in_unit(dalton)
        CG = 0
        CGC = 0
        if str(key[0]) in exclusionlist:
            INE = list(exclusionlist[str(key[0])])
        else:
            INE = list()
        INE14 = list()
        epsilon = float(force.epsilon.value_in_unit(kilojoule_per_mole))
        rmin = 2 * force.rmin_half.value_in_unit(nanometer)
        C6 = 2 * epsilon * (rmin**6)
        C12 = epsilon * (rmin**12)
        IACname = force.id
        print(force.smirks)
        top.add_new_SOLUTEATOM(ATNM=ATNM, MRES=MRES, IAC=IAC, MASS=MASS, CG=CG, CGC=CGC, INE=INE, INE14=INE14, C6=C6, C12=C12, IACname=IACname)
    moleculeItr += 1 

In [ ]:
len(moleculeFF.name)

In [ ]:
if len(moleculeFF.name) >= 1:
    top.add_new_resname(moleculeFF.name)
else:
    top.add_new_resname(moleculeFF.hill_formula)

In [ ]:
top.TEMPERATUREGROUPS.content

In [ ]:
moleculeFF.n_atoms

In [ ]:
top.make_ordered()

In [ ]:
print(top)

In [ ]:
top2 = Top(input=data.data_dir+"/topology_templates/blank_template.top")

In [ ]:
top3 = openforcefield2gromos.openforcefield2gromos(moleculeFF, gromosTop=top2, forcefield_name=data.data_dir + "/ff/SMIRNOFF/frosst_12-12-18.xml").convert_return()

In [ ]:
top3.make_ordered()

In [ ]:
print(top3)

In [ ]:
from pygromos.gromos import gromosPP, gromosXX

In [ ]:
gromosPP.GromosPP(bin_path="/home/mlehner/gromosPlsPls/gromos++/BUILD/programs")

In [ ]:
top3._orig_file_path = "/home/mlehner/Documents/dev/temp19/top3.top"

In [ ]:
top3.write("/home/mlehner/Documents/dev/temp19/top3.top")

In [ ]:
top4 = openforcefield2gromos(moleculeFF).convert_return()

In [ ]:
top4.make_ordered()

In [ ]:
print(top4)

In [ ]:
top5 = openforcefield2gromos(moleculeFF, gromosTop=Top(input=None)).convert_return()

In [ ]:
top5.make_ordered()

In [ ]:
print(top5)

In [ ]:
top5.write("/home/mlehner/Documents/dev/test2.top")

In [ ]:
mol2=AllChem.MolFromSmarts("[C:1]=C(C)C(=O)OC")

In [ ]:
mol2

In [ ]:
for a in mol2.GetAtoms():
    print(a.GetAtomicNum())
    print(a.GetIdx())
    print(a.GetTotalNumHs())
    print(a.GetAtomMapNum())
    print("----------------")

In [ ]:
AllChem.MolFromSmarts("[#16][#6]=[#6:1]([#6])[#8][#6]")